## TASK 4

In [8]:
source("./Imports.R")

## Description
Select 7 columns: DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes from given PostsCounts table:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;JOIN:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ALL from given Answers table:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;AnswersNumber, OwnerUserId from table Posts  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Takes only Posts where PostIdType is 2  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Groups the output by OwnerUsedId  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;AND  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ALL from given Questions table:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;AnswersNumber, OwnerUserId from table Posts  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Takes only Posts where PostIdType is 2  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Groups the output by OwnerUsedId  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ON ownerUserId field  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Takes rows where AnswersNumber > QuestionsNumber  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Sorts the data by AnswersNumber (Descending)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Takes first 5 outputs  
Joins PostsCounts table and Users table by user id  

### Effectively returns top 5 Users with highest AnswersNumber, but with lower QuestionsNumber that AnswersNumber


In [2]:
sql_4 <- function(Posts, Users) {
  sqldf("SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
        FROM (
                SELECT *
                FROM (
                        SELECT COUNT(*) as AnswersNumber, OwnerUserId
                        FROM Posts
                        WHERE PostTypeId = 2
                        GROUP BY OwnerUserId
                    ) AS Answers
                JOIN
                    (
                        SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                        FROM Posts
                        WHERE PostTypeId = 1
                        GROUP BY OwnerUserId
                    ) AS Questions
                ON Answers.OwnerUserId = Questions.OwnerUserId
                WHERE AnswersNumber > QuestionsNumber
                ORDER BY AnswersNumber DESC
                LIMIT 5
            ) AS PostsCounts
        JOIN Users
        ON PostsCounts.OwnerUserId = Users.Id")
}

In [3]:
base_4 <- function(Posts, Users) {
    answers_count <- aggregate(Posts$Id[Posts$PostTypeId == 2], 
                               by = list(OwnerUserId = Posts$OwnerUserId[Posts$PostTypeId == 2]), 
                               FUN = length)
    answers_count <- answers_count[order(c(2,1))]
    colnames(answers_count) <- c("AnswersNumber","OwnerUserId")
    questions_count <- aggregate(Posts$Id[Posts$PostTypeId == 1], 
                                 by = list(OwnerUserId = Posts$OwnerUserId[Posts$PostTypeId == 1]), 
                                 FUN = length)
    questions_count <- questions_count[order(c(2,1))]
    colnames(questions_count) <- c("QuestionsNumber","OwnerUserId")
    merged_counts <- merge(answers_count, questions_count, by = "OwnerUserId")
    merged_counts <- merged_counts[merged_counts$AnswersNumber > merged_counts$QuestionsNumber, ]
    merged_counts_users <- merge(merged_counts, Users, by.x = "OwnerUserId", by.y = "Id")
    result <- merged_counts_users[, c("DisplayName", "QuestionsNumber", "AnswersNumber", "Location", "Reputation", "UpVotes", "DownVotes")]
    result <- result[order(-result$AnswersNumber), ][1:5,]
    return(result)
}


In [4]:
dplyr_4 <- function(Posts,Users){
    PostsCounts <- Posts %>% 
    filter(PostTypeId == 2) %>% 
    group_by(OwnerUserId) %>% 
    summarise(AnswersNumber = n()) %>% 
    inner_join(Posts %>% 
                filter(PostTypeId == 1) %>% 
                group_by(OwnerUserId) %>% 
                summarise(QuestionsNumber = n()), by = "OwnerUserId") %>%
    na.omit %>%
    filter(AnswersNumber > QuestionsNumber) %>% 
    arrange(desc(AnswersNumber)) %>% 
    head(5) %>% 
    select(OwnerUserId, QuestionsNumber, AnswersNumber)
    as.data.frame(
        Users %>% 
            inner_join(PostsCounts, by = c("Id" = "OwnerUserId")) %>% 
            select(DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes)
    )
}

In [5]:
table_4 <- function(Posts,Users){
    setDT(Posts)
    setDT(Users)
    answers_df <- Posts[PostTypeId == 2, .(AnswersNumber = .N), by = OwnerUserId]
    questions_df <- Posts[PostTypeId == 1, .(QuestionsNumber = .N), by = OwnerUserId]
    merged_df <- na.omit(merge(answers_df,questions_df))
    merge(merged_df[AnswersNumber > QuestionsNumber][order(-AnswersNumber)][1:5], Users,by.x = "OwnerUserId", by.y = "Id")[,.(DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes)]
}

In [6]:
comp(
    sql_4(Posts,Users),
    base_4(Posts, Users),
    dplyr_4(Posts, Users),
    table_4(Posts, Users)
)

[1] "COMPARING GIVEN OUTPUTS TO SQL ONE: "
[1] "BASE: TRUE"
[1] "DPLYR: TRUE  permuted"
[1] "TABLE: TRUE  permuted"


In [7]:
table_4(Posts, Users)

DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
<chr>,<int>,<int>,<chr>,<int>,<int>,<int>
Mark Mayo,342,1950,"Christchurch, New Zealand",153872,15817,1613
lambshaanxy,104,1394,,99487,13629,253
Relaxed,8,1414,,91865,10911,570
phoog,12,1602,"New York, NY",111746,8212,529
Crazydre,171,1163,,72332,2259,132
